In [14]:
import time
from Crypto.PublicKey import RSA
from Crypto.Cipher import AES, PKCS1_OAEP
from Crypto.Random import get_random_bytes
from tinyec import registry
import hashlib
import secrets

In [15]:
def generate_ecc_key_pair():
    print("\n[STEP 1: ECC Key Pair Generation]")
    curve = registry.get_curve('brainpoolP256r1')
    print(f"ECC Curve: {curve.name}")
    private_key = secrets.randbelow(curve.field.n)
    public_key = private_key * curve.g
    print(f"ECC Private Key: {private_key}")
    print(f"ECC Public Key: ({public_key.x}, {public_key.y})")
    return private_key, public_key

In [16]:
def generate_rsa_key_pair():
    print("\n[STEP 2: RSA Key Pair Generation]")
    key = RSA.generate(2048)
    private_key = key.export_key()
    public_key = key.publickey().export_key()
    print(f"Generated RSA Modulus (n): {key.n}")
    print(f"Generated RSA Public Exponent (e): {key.e}")
    print(f"Generated RSA Private Exponent (d): {key.d}")
    print(f"Truncated RSA Private Key: {private_key.decode()[:50]}...")
    print(f"Truncated RSA Public Key: {public_key.decode()[:50]}...")
    return private_key, public_key

In [17]:
def ecc_point_to_256_bit_key(point):
    print("\n[STEP 3: ECC Shared Key Derivation]")
    print(f"ECC Shared Point Coordinates: ({point.x}, {point.y})")
    sha = hashlib.sha256(int.to_bytes(point.x, 32, 'big'))
    sha.update(int.to_bytes(point.y, 32, 'big'))
    derived_key = sha.digest()
    print(f"Derived ECC 256-bit Key: {derived_key.hex()}")
    return derived_key

In [18]:
def encrypt_message(msg, ecc_public_key, rsa_public_key):
    print("\n[STEP 4: Message Encryption]")

    # ECC Key Exchange
    print("\n4.1: Performing ECC Key Exchange...")
    curve = registry.get_curve('brainpoolP256r1')
    ecc_private_key = secrets.randbelow(curve.field.n)
    print(f"Ephemeral ECC Private Key: {ecc_private_key}")
    shared_ecc_key = ecc_private_key * ecc_public_key
    secret_key = ecc_point_to_256_bit_key(shared_ecc_key)

    # AES Encryption
    print("\n4.2: Encrypting Message with AES...")
    aes_key = secret_key  # Use ECC-derived key as AES key
    print(f"Using ECC-Derived AES Key: {aes_key.hex()}")
    cipher_aes = AES.new(aes_key, AES.MODE_GCM)
    ciphertext, tag = cipher_aes.encrypt_and_digest(msg.encode('utf-8'))
    print(f"AES Nonce: {cipher_aes.nonce.hex()}")
    print(f"AES Ciphertext: {ciphertext.hex()}")
    print(f"AES Tag: {tag.hex()}")

    # RSA Encryption of AES Key
    print("\n4.3: Encrypting AES Key with RSA...")
    cipher_rsa = PKCS1_OAEP.new(RSA.import_key(rsa_public_key))
    enc_aes_key = cipher_rsa.encrypt(aes_key)
    print(f"RSA Encrypted AES Key: {enc_aes_key.hex()}")

    return (cipher_aes.nonce, tag, ciphertext, enc_aes_key)

In [19]:
def decrypt_message(enc_msg, ecc_private_key, rsa_private_key):
    print("\n[STEP 5: Message Decryption]")

    # ECC Key Exchange
    print("\n5.1: Performing ECC Key Exchange...")
    nonce, tag, ciphertext, enc_aes_key = enc_msg

    # RSA Decryption of AES Key
    print("\n5.2: Decrypting AES Key with RSA...")
    cipher_rsa = PKCS1_OAEP.new(RSA.import_key(rsa_private_key))
    aes_key = cipher_rsa.decrypt(enc_aes_key)
    print(f"Decrypted AES Key (from RSA): {aes_key.hex()}")

    # AES Decryption
    print("\n5.3: Decrypting Message with AES...")
    cipher_aes = AES.new(aes_key, AES.MODE_GCM, nonce=nonce)
    plaintext = cipher_aes.decrypt_and_verify(ciphertext, tag)
    print(f"Decrypted Plaintext: {plaintext.decode('utf-8')}")

    return plaintext.decode('utf-8')

In [20]:
# Example
print("[STEP 0: Key Generation Phase]")
time.sleep(1)
ecc_private_key, ecc_public_key = generate_ecc_key_pair()
rsa_private_key, rsa_public_key = generate_rsa_key_pair()

message = input("\nEnter a message to encrypt: ")

# Encryption process
encrypted_msg = encrypt_message(message, ecc_public_key, rsa_public_key)

# Decryption process
decrypted_msg = decrypt_message(encrypted_msg, ecc_private_key, rsa_private_key)

print("\n[FINAL RESULTS]")
print(f"Original Message: {message}")
print(f"Decrypted Message: {decrypted_msg}")

[STEP 0: Key Generation Phase]

[STEP 1: ECC Key Pair Generation]
ECC Curve: brainpoolP256r1
ECC Private Key: 23575382442678294219125130071623757134030501071268970277470836998739179510555
ECC Public Key: (688716717198379682183050685792434455170761694714850365798094516035944166739, 12997550443075639612454433480770298632582368132364651893653302398387108269160)

[STEP 2: RSA Key Pair Generation]
Generated RSA Modulus (n): 2396825523118161141072462795314400712877130513746494250326844181738392501281026213904526894048931116367058046413190295964251160834704430027933083995322891968522851337205417705557039997856162281452874176350526857780796059730139318229516648834758502578716163112421070055792415182018406970702250489177642199655826054022903158437927559147037504596264690680167265653770105558029192284786220457263104282695489535687332647402852574869869586114255524916629588953193049406949382986601616450389082032303232442472341210033647806858273937335690797447720983294292005348616223168599299942565